In [1]:
import pandas as pd
from itertools import combinations

# Load the input file with encoding specified to handle special characters
input_file = r"C:\Users\dinesh\OneDrive\Desktop\Sampledata.csv"
data = pd.read_csv(input_file, encoding='ISO-8859-1')

# Define the output file path to save results on the desktop
output_file = r"C:\Users\dinesh\OneDrive\Desktop\sampledata.txt"
with open(output_file, 'w') as file:

    # Prompt for Primary Keys
    primary_key = input("Enter primary key(s), separated by commas: ").split(',')
    primary_key = [key.strip() for key in primary_key]

    # Prompt for Functional Dependencies
    FDs = {}
    print("Enter Functional Dependencies in format 'A,B -> C,D' (type 'done' when finished):")
    while True:
        fd_input = input()
        if fd_input.lower() == 'done':
            break
        try:
            det, dep = fd_input.split("->")
            det_attrs = [attr.strip() for attr in det.split(',')]
            dep_attrs = [attr.strip() for attr in dep.split(',')]
            FDs[tuple(det_attrs)] = dep_attrs
            file.write(f"{det} -> {dep}\n")
        except ValueError:
            print("Invalid format. Please enter in 'A,B -> C,D' format.")

    # Prompt for Multi-Valued Dependencies (MVDs)
    MVDs = {}
    print("Enter Multi-Valued Dependencies in format 'A ->> B' (type 'done' when finished):")
    while True:
        mvd_input = input()
        if mvd_input.lower() == 'done':
            break
        try:
            det, dep = mvd_input.split("->>")
            det = det.strip()
            dep = dep.strip()
            if det in MVDs:
                MVDs[det].append(dep)
            else:
                MVDs[det] = [dep]
            file.write(f"Multi-Valued Dependency: {det} ->> {dep}\n")
        except ValueError:
            print("Invalid format. Please enter in 'A ->> B' format.")

    # Get normalization level from the user
    max_normal_form = input("Select the highest normal form to normalize (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")
    if max_normal_form in ["1", "2", "3", "4", "5"]:
        max_normal_form = int(max_normal_form)

    # Prompt for Join Dependencies (JDs) only if target form is 5NF
    JDs = []
    if max_normal_form == 5:
        print("Enter Join Dependencies in format 'A,B' (type 'done' when finished):")
        while True:
            jd_input = input()
            if jd_input.lower() == 'done':
                break
            jd = [attr.strip() for attr in jd_input.split(',')]
            JDs.append(jd)
            file.write(f"Join Dependency: {', '.join(jd)}\n")

    # Function to check and convert 1NF
    def check_and_convert_1NF(df):
        non_atomic_columns = []
        for col in df.columns:
            if df[col].apply(lambda x: isinstance(x, list)).any():
                non_atomic_columns.append(col)
                df = df.explode(col)
        return df, non_atomic_columns

    # Function to check 2NF
    def check_2NF(df, primary_key, FDs):
        non_prime_attrs = [col for col in df.columns if col not in primary_key]
        for det, dep in FDs.items():
            if set(det).issubset(primary_key) and any(attr in non_prime_attrs for attr in dep):
                return False
        return True

    # Function to check 3NF
    def check_3NF(df, primary_key, FDs):
        for det, dep in FDs.items():
            if set(det) != set(primary_key) and not set(dep).issubset(primary_key):
                return False
        return True

    # Function to check BCNF
    def check_BCNF(df, primary_key, FDs):
        for det, dep in FDs.items():
            if not set(det).issubset(primary_key):
                return False
        return True

    # Function to check and decompose 4NF based on MVDs
    def check_and_decompose_4NF(df, MVDs):
        decomposed_tables = []
        main_table_columns = list(df.columns)

        for det, dep_list in MVDs.items():
            for dep in dep_list:
                decomposed_table = df[[det, dep]].drop_duplicates()
                decomposed_tables.append((decomposed_table, [det, dep]))  # Track primary key for decomposed table
                if dep in main_table_columns:
                    main_table_columns.remove(dep)

                file.write(f"4NF Decomposition: Created separate table with {det} ->> {dep}\n")

        main_table = df[main_table_columns].drop_duplicates()
        return [(main_table, primary_key)] + decomposed_tables  # Include main table with original primary key

    # Function to check and decompose 5NF based on JDs
    def check_and_decompose_5NF(df, JDs):
        decomposed_tables = []

        for jd in JDs:
            if set(jd).issubset(df.columns):
                decomposed_table = df[jd].drop_duplicates()
                decomposed_tables.append((decomposed_table, jd))
                file.write(f"5NF Decomposition: Created separate table for JD ({', '.join(jd)})\n")

        if not decomposed_tables:
            file.write("No 5NF decomposition required.\n")
        return [(df, primary_key)] + decomposed_tables  # Include main table with original primary key

    # SQL Query Generator and Schema Representation
    def generate_sql_and_schema(table_name, df, primary_keys):
        sql_query = f"CREATE TABLE {table_name} (\n"
        schema_representation = f"Table: {table_name}\nAttributes:\n"

        for col, dtype in zip(df.columns, df.dtypes):
            sql_type = 'INT' if 'int' in str(dtype) else 'VARCHAR(255)'
            sql_query += f"  {col} {sql_type},\n"
            schema_representation += f"  - {col} ({sql_type})\n"

        primary_keys_str = ", ".join(primary_keys)
        sql_query += f"  PRIMARY KEY ({primary_keys_str})\n);"
        schema_representation += f"Primary Key(s): {primary_keys_str}\n\n"

        # Print and write to file
        print(sql_query)
        file.write(sql_query + "\n\n" + schema_representation)

    # Main Normalization Process
    def normalize_database(df, primary_key, FDs, MVDs, max_normal_form, JDs=[]):
        df, non_atomic_columns = check_and_convert_1NF(df)
        file.write("1NF Applied\n" if not non_atomic_columns else f"Converted columns to atomic: {non_atomic_columns}\n")
        generate_sql_and_schema('Table_1NF', df, primary_key)

        highest_nf = 1
        if max_normal_form >= 2 and check_2NF(df, primary_key, FDs):
            file.write("2NF satisfied.\n")
            highest_nf = 2
        else:
            file.write("2NF decomposition required.\n")

        if max_normal_form >= 3 and check_3NF(df, primary_key, FDs):
            file.write("3NF satisfied.\n")
            highest_nf = 3
        else:
            file.write("3NF decomposition required.\n")

        if max_normal_form >= 4 and check_BCNF(df, primary_key, FDs):
            file.write("BCNF satisfied.\n")
            highest_nf = "BCNF"
        else:
            file.write("BCNF decomposition required.\n")

        decomposed_tables = [(df, primary_key)]
        if max_normal_form >= 4:
            decomposed_tables = check_and_decompose_4NF(df, MVDs)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_4NF_{i}"
                generate_sql_and_schema(table_name, table, keys)
            highest_nf = 4 if highest_nf != "BCNF" else highest_nf

        if max_normal_form == 5:
            decomposed_tables = check_and_decompose_5NF(df, JDs)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_5NF_{i}"
                generate_sql_and_schema(table_name, table, keys)

        file.write(f"The highest normal form achieved for this relation is: {highest_nf}\n")

    # Execute the normalization
    normalize_database(data, primary_key, FDs, MVDs, max_normal_form, JDs)

print(f"Normalization results saved to {output_file}")

Enter primary key(s), separated by commas: OrderID, CustomerID, DrinkID, FoodID
Enter Functional Dependencies in format 'A,B -> C,D' (type 'done' when finished):
OrderID -> CustomerID
Done
Enter Multi-Valued Dependencies in format 'A ->> B' (type 'done' when finished):
OrderID ->> DrinkID
Done
Select the highest normal form to normalize (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): 5
Enter Join Dependencies in format 'A,B' (type 'done' when finished):
OrderID,CustomerID OrderID,DrinkID OrderID,FoodID
Done
CREATE TABLE Table_1NF (
  OrderID INT,
  Date VARCHAR(255),
  PromocodeUsed VARCHAR(255),
  TotalCost VARCHAR(255),
  TotalDrinkCost VARCHAR(255),
  TotalFoodCost VARCHAR(255),
  CustomerID INT,
  CustomerName VARCHAR(255),
  DrinkID INT,
  DrinkName VARCHAR(255),
  DrinkSize VARCHAR(255),
  DrinkQuantity INT,
  Milk VARCHAR(255),
  DrinkIngredient VARCHAR(255),
  DrinkAllergen VARCHAR(255),
  FoodID INT,
  FoodName VARCHAR(255),
  FoodQuantity INT,
  FoodIngredient VARCHAR(255)